In [1]:
import findspark
import os
spark_home = os.path.abspath("/Users/yanji/opt/spark")
findspark.init(spark_home)
import pyspark

os.system("kill -9 $(lsof -i:4050 -t) 2> /dev/null")

spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("test") \
    .config("spark.driver.bindAddress","localhost") \
    .config("spark.ui.port","4050") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

print("http://{}:{}".format(spark.conf.get("spark.driver.bindAddress"), spark.conf.get("spark.ui.port")))


22/05/19 18:58:04 WARN Utils: Your hostname, YandeMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.164 instead (on interface en0)
22/05/19 18:58:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 18:58:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


http://localhost:4050


## DataFrames and Datasets
### Schemas

In essence, within the Structured APIs, there are two more APIs, the “untyped” DataFrames and the “typed” Datasets. To say that DataFrames are untyped is aslightly inaccurate; they have types, but Spark maintains them completely and only checks whether those types line up to those specified in the schema at runtime. Datasets, on the other hand, check whether types conform to the specification at compile time. Datasets are only available to Java Virtual Machine (JVM)–based languages (Scala and Java) and we specify types with case classes or Java beans.

For the most part, you’re likely to work with DataFrames. To Spark (in Scala), DataFrames are simply Datasets of Type Row. The “Row” type is Spark’s internal representation of its optimized in-memory format for computation. This format makes for highly specialized and efficient computation because rather than using JVM types, which can cause high garbage-collection and object instantiation costs, Spark can operate on its own internal format without incurring any of those costs. To Spark (in Python or R), there is no such thing as a Dataset: everything is a DataFrame and therefore we always operate on that optimized format.

In [2]:
df = spark.range(500).toDF("number")
df.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [3]:
df.select(df["number"] + 10).show()

+-------------+
|(number + 10)|
+-------------+
|           10|
|           11|
|           12|
|           13|
|           14|
|           15|
|           16|
|           17|
|           18|
|           19|
|           20|
|           21|
|           22|
|           23|
|           24|
|           25|
|           26|
|           27|
|           28|
|           29|
+-------------+
only showing top 20 rows



In [4]:
from pyspark.sql.types import ByteType
b = ByteType()

codes/sql -> logical plan -> physical plan -> executes the physical plan on the cluster